In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanicdataset-traincsv/train.csv


In [2]:
df = pd.read_csv('/kaggle/input/titanicdataset-traincsv/train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.drop(columns=['PassengerId','Name', 'Ticket', 'Cabin'], inplace=True)

In [5]:
df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train,X_test,Y_train,Y_test= train_test_split(df.drop(columns=['Survived']), df['Survived'],test_size=0.2, random_state=40)

In [8]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
661,3,male,40.0,0,0,7.2250,C
393,1,female,23.0,1,0,113.2750,C
266,3,male,16.0,4,1,39.6875,S
144,2,male,18.0,0,0,11.5000,S
446,2,female,13.0,0,1,19.5000,S
...,...,...,...,...,...,...,...
440,2,female,45.0,1,1,26.2500,S
165,3,male,9.0,0,2,20.5250,S
7,3,male,2.0,3,1,21.0750,S
219,2,male,30.0,0,0,10.5000,S


In [9]:
X_test

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
246,3,female,25.0,0,0,7.7750,S
588,3,male,22.0,0,0,8.0500,S
472,2,female,33.0,1,2,27.7500,S
71,3,female,16.0,5,2,46.9000,S
654,3,female,18.0,0,0,6.7500,Q
...,...,...,...,...,...,...,...
284,1,male,NaN,0,0,26.0000,S
727,3,female,NaN,0,0,7.7375,Q
599,1,male,49.0,1,0,56.9292,C
525,3,male,40.5,0,0,7.7500,Q


In [10]:
Y_test

246    0
588    0
472    1
71     0
654    0
      ..
284    0
727    1
599    1
525    0
457    1
Name: Survived, Length: 179, dtype: int64

In [11]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age         139
SibSp         0
Parch         0
Fare          0
Embarked      1
dtype: int64

In [12]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [13]:
#Imputation Transformer
tnf1 = ColumnTransformer(
    [
        ('impute_age', SimpleImputer(), [2]),
        ('impute_embarked', SimpleImputer(strategy='most_frequent'),[6])
    ], remainder='passthrough'
)

In [14]:
#OneHot Encoding
tnf2 = ColumnTransformer(
    [
        ('OHE_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1,6])
    ], remainder = 'passthrough'
)

In [15]:
#Scaling The Values
from sklearn.preprocessing import MinMaxScaler

tnf3 = ColumnTransformer(
    [
        ('scale', MinMaxScaler(), slice(1,10))
    ]
) 

In [16]:
#Feature selection
from sklearn.feature_selection import SelectKBest,chi2
tnf4 = SelectKBest(score_func=chi2,k=8)


In [17]:
from sklearn.tree import DecisionTreeClassifier

tnf5 = DecisionTreeClassifier()

# Create pipelines

In [18]:
from sklearn.pipeline import Pipeline,make_pipeline
pipe = make_pipeline(tnf1,tnf2,tnf3,tnf4,tnf5)

# Alternate syntax


In [19]:
'''pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])'''

"pipe = Pipeline([\n    ('trf1',trf1),\n    ('trf2',trf2),\n    ('trf3',trf3),\n    ('trf4',trf4),\n    ('trf5',trf5)\n])"

In [20]:
pipe.fit(X_train,Y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('OHE_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(1, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7ea4ca298430>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [21]:
# Display Pipeline

from sklearn import set_config
set_config(display='diagram')

In [22]:
#prediction
y_pred= pipe.predict(X_test)

In [23]:
y_pred

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 0])

In [24]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,y_pred)

0.5865921787709497

In [25]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, Y_train, cv=5, scoring='accuracy').mean()

0.6123707278636856

In [26]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))